In [ ]:
#PhoneBook.py

import pygame
import pandas as pd
import tkinter as tk
from tkinter import filedialog
from pygame.locals import *

# Initialize pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 600, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("PhoneBook Application")

# Font for text rendering
font = pygame.font.SysFont('Arial', 20)
font_large = pygame.font.SysFont('Arial', 25)

# Button dimensions
BUTTON_WIDTH, BUTTON_HEIGHT = 200, 40
BUTTON_COLOR = (70, 130, 180)
BUTTON_HOVER_COLOR = (100, 150, 200)

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Set initial state for the DataFrame (PhoneBook data)
data = pd.DataFrame(columns=['Username', 'Phone Number', 'First Name', 'Second Name'])

# Button class for creating and handling buttons in the interface
class Button:
    def __init__(self, x, y, width, height, text, action=None):
        self.rect = pygame.Rect(x, y, width, height)  # Define button position and size
        self.text = text  # The label for the button
        self.action = action  # The function to call when the button is clicked

    # Function to draw the button and handle hover effect
    def draw(self, screen):
        mouse_x, mouse_y = pygame.mouse.get_pos()  # Get current mouse position
        if self.rect.collidepoint(mouse_x, mouse_y):  # Check if mouse is over the button
            pygame.draw.rect(screen, BUTTON_HOVER_COLOR, self.rect)  # Hover effect
        else:
            pygame.draw.rect(screen, BUTTON_COLOR, self.rect)  # Normal button color

        # Render the button's text and center it inside the button
        text_surface = font.render(self.text, True, WHITE)
        text_rect = text_surface.get_rect(center=self.rect.center)
        screen.blit(text_surface, text_rect)

    # Function to check for a click event on the button
    def click(self):
        mouse_x, mouse_y = pygame.mouse.get_pos()
        if self.rect.collidepoint(mouse_x, mouse_y):  # If the mouse clicked on the button
            if self.action:  # If the button has an associated action
                self.action()

# Function to load CSV data using tkinter.filedialog
def load_csv():
    global data
    filename = str(input("Insert the path to csv file:"))  # Get file path from user input

    try:
        new_data = pd.read_csv(filename)  # Load the new data
        data = pd.concat([data, new_data], ignore_index=True)  # Append instead of replacing
        #data = data.drop_duplicates(subset='Username', keep='last')  # Keep the most recent version
        print(f"Appended data from {filename}")
    except Exception as e:
        print(f"Failed to load CSV file: {e}")

# Function to add a new person to the phonebook
def add_person():
    global data
    username = input("Enter Username: ")
    phone = input("Enter Phone Number: ")
    first_name = input("Enter First Name: ")
    second_name = input("Enter Second Name: ")

    # Create a new DataFrame entry for the new person
    new_entry = pd.DataFrame([[username, phone, first_name, second_name]],
                             columns=['Username', 'Phone Number', 'First Name', 'Second Name'])
    data = pd.concat([data, new_entry], ignore_index=True)  # Add the new entry to the main DataFrame

# Function to edit an existing person's details
def edit_person():
    global data
    username = input("Enter the Username of the person to edit: ")
    if username in data['Username'].values:  # Check if username exists in the data
        person = data[data['Username'] == username].iloc[0]  # Get the person details
        # Prompt the user to edit details
        new_phone = input(f"Edit Phone Number (current: {person['Phone Number']}): ")
        new_first_name = input(f"Edit First Name (current: {person['First Name']}): ")
        new_second_name = input(f"Edit Second Name (current: {person['Second Name']}): ")

        # Update the DataFrame with the new details
        data.loc[data['Username'] == username, 'Phone Number'] = new_phone
        data.loc[data['Username'] == username, 'First Name'] = new_first_name
        data.loc[data['Username'] == username, 'Second Name'] = new_second_name
    else:
        print("Username not found.")  # Inform the user if the username doesn't exist

# Function to delete a person from the phonebook
def delete_person():
    global data
    username = input("Enter the Username of the person to delete: ")
    if username in data['Username'].values:  # Check if username exists in the data
        data = data[data['Username'] != username]  # Remove the person from the DataFrame
    else:
        print("Username not found.")  # Inform the user if the username doesn't exist

# Function to display the phonebook data table on the screen
def display_table():
    screen.fill(WHITE)  # Fill the screen with white color
    y_offset = 60  # Start the table below the title

    # Define the header for the table
    header = "Username | Phone Number | First Name | Second Name"
    header_surface = font_large.render(header, True, BLACK)
    screen.blit(header_surface, (50, y_offset))

    y_offset += 40
    pygame.draw.line(screen, BLACK, (50, y_offset), (WIDTH-50, y_offset), 2)  # Draw a line under the header

    # Loop through each entry in the data and display it
    for idx, row in data.iterrows():
        y_offset += 30  # Move to the next row position
        text = f"{row['Username']} | {row['Phone Number']} | {row['First Name']} | {row['Second Name']}"
        text_surface = font.render(text, True, BLACK)
        screen.blit(text_surface, (50, y_offset))

# Creating buttons for different actions (Upload CSV, Add Person, Edit Person, Delete Person)
buttons = [
    Button(50, 300, BUTTON_WIDTH, BUTTON_HEIGHT, "Upload CSV", load_csv),
    Button(270, 300, BUTTON_WIDTH, BUTTON_HEIGHT, "Add Person", add_person),
    Button(50, 350, BUTTON_WIDTH, BUTTON_HEIGHT, "Edit Person", edit_person),
    Button(270, 350, BUTTON_WIDTH, BUTTON_HEIGHT, "Delete Person", delete_person)
]

# Main loop for handling events and updating the display
running = True
while running:
    for event in pygame.event.get():
        if event.type == QUIT:  # Check if the window is closed
            running = False
        elif event.type == MOUSEBUTTONDOWN:  # Check if a mouse button is pressed
            for button in buttons:
                button.click()  # Check if any button is clicked

    # Draw the table and buttons on the screen
    display_table()
    for button in buttons:
        button.draw(screen)  # Draw each button

    pygame.display.flip()  # Update the display

pygame.quit()  # Quit pygame when the loop ends


In [ ]:
#datas.csv

Username,Phone Number,First Name,Second Name
beka,Bekassyl,Zatybek,87007007070
ers,Ersultan,Serik,87778887788
dias,Dias,Serik,800011001010

In [ ]:
#Snake

import pygame
import sqlite3
import random

# ------------------------- DATABASE SETUP -------------------------

# Initialize the database and create necessary tables if they don't exist
def init_db():
    conn = sqlite3.connect("snake_game.db")
    cur = conn.cursor()

    # Create a user table to store usernames
    cur.execute('''
        CREATE TABLE IF NOT EXISTS user (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE NOT NULL
        )
    ''')

    # Create a table to store user scores and levels
    cur.execute('''
        CREATE TABLE IF NOT EXISTS user_score (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            level INTEGER,
            score INTEGER,
            FOREIGN KEY(user_id) REFERENCES user(id)
        )
    ''')

    conn.commit()
    conn.close()

# Fetch user by username or create a new user entry if not exists
def get_or_create_user(username):
    conn = sqlite3.connect("snake_game.db")
    cur = conn.cursor()

    # Check if the user exists
    cur.execute("SELECT id FROM user WHERE username=?", (username,))
    user = cur.fetchone()

    # Get user_id if user exists, otherwise insert new user
    if user:
        user_id = user[0]
    else:
        cur.execute("INSERT INTO user (username) VALUES (?)", (username,))
        conn.commit()
        user_id = cur.lastrowid

    # Fetch the highest level achieved by the user
    cur.execute("SELECT MAX(level) FROM user_score WHERE user_id=?", (user_id,))
    level = cur.fetchone()[0] or 1  # Default to level 1 if none found

    conn.close()
    return user_id, level

# Save the score and level for a user
def save_score(user_id, level, score):
    conn = sqlite3.connect("snake_game.db")
    cur = conn.cursor()
    cur.execute("INSERT INTO user_score (user_id, level, score) VALUES (?, ?, ?)",
                (user_id, level, score))
    conn.commit()
    conn.close()

# ------------------------- GAME LOGIC -------------------------

# Render text on the screen
def draw_text(screen, text, x, y, font):
    screen.blit(font.render(text, True, (255, 255, 255)), (x, y))

# Generate wall positions based on current level
def generate_walls(level, width, height, tile_size):
    walls = []
    for _ in range(level):  # One more wall for each level
        x = random.randint(1, width // tile_size - 2)
        y = random.randint(1, height // tile_size - 2)
        walls.append((x, y))
    return walls

# Display the start menu with the user's name and current level
def show_start_menu(screen, font, username, level):
    button_rect = pygame.Rect(250, 200, 100, 40)
    waiting = True

    while waiting:
        screen.fill((30, 30, 30))
        draw_text(screen, f"Welcome, {username}!", 200, 100, font)
        draw_text(screen, f"Current Level: {level}", 220, 140, font)

        # Draw the Start button
        pygame.draw.rect(screen, (70, 130, 180), button_rect)
        draw_text(screen, "Start", button_rect.x + 25, button_rect.y + 10, font)

        pygame.display.flip()

        # Wait for user to click the Start button or quit
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if button_rect.collidepoint(event.pos):
                    waiting = False

# Update user's current level in the database
def update_user_level(user_id, new_level):
    conn = sqlite3.connect("snake_game.db")
    cur = conn.cursor()
    cur.execute("UPDATE user SET current_level = ? WHERE id = ?", (new_level, user_id))
    conn.commit()
    conn.close()

# Ensure the `current_level` column exists in the user table
def update_db_schema():
    conn = sqlite3.connect("snake_game.db")
    cur = conn.cursor()
    try:
        cur.execute("ALTER TABLE user ADD COLUMN current_level INTEGER DEFAULT 1")
        conn.commit()
    except sqlite3.OperationalError:
        pass  # If column already exists, ignore the error
    conn.close()

# ------------------------- MAIN GAME LOOP -------------------------
def main(username):
    WIDTH, HEIGHT = 600, 400
    TILE_SIZE = 20

    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Snake Game with Levels")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont(None, 30)

    # Ensure DB schema is updated and get user data
    update_db_schema()
    user_id, level = get_or_create_user(username)

    # Show start menu
    show_start_menu(screen, font, username, level)

    # Short "get ready" message
    screen.fill((0, 0, 0))
    draw_text(screen, f"Welcome, {username}!", WIDTH // 2 - 80, HEIGHT // 2 - 30, font)
    draw_text(screen, f"Level {level} - Get Ready!", WIDTH // 2 - 100, HEIGHT // 2, font)
    pygame.display.flip()
    pygame.time.wait(1000)

    # Game variables
    score = 0
    speed = 8 + level * 4
    walls = generate_walls(level, WIDTH, HEIGHT, TILE_SIZE)
    snake = [(5, 5)]
    direction = (1, 0)  # Moving right
    food = (random.randint(0, WIDTH // TILE_SIZE - 1), random.randint(0, HEIGHT // TILE_SIZE - 1))

    paused = False
    running = True
    game_over = False

    # Display "Get Ready" before game starts
    screen.fill((0, 0, 0))
    draw_text(screen, f"Welcome, {username}!", WIDTH // 2 - 80, HEIGHT // 2 - 30, font)
    draw_text(screen, f"Level {level} - Get Ready!", WIDTH // 2 - 100, HEIGHT // 2, font)
    pygame.display.flip()
    pygame.time.wait(3000)

    # Main game loop
    try:
        while running:
            screen.fill((0, 0, 0))
            draw_text(screen, f"User: {username} | Level: {level} | Score: {score}", 10, 10, font)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    save_score(user_id, level, score)
                    running = False
                elif event.type == pygame.KEYDOWN:
                    # Change direction
                    if event.key == pygame.K_UP and direction != (0, 1): direction = (0, -1)
                    elif event.key == pygame.K_DOWN and direction != (0, -1): direction = (0, 1)
                    elif event.key == pygame.K_LEFT and direction != (1, 0): direction = (-1, 0)
                    elif event.key == pygame.K_RIGHT and direction != (-1, 0): direction = (1, 0)
                    elif event.key == pygame.K_p:
                        paused = not paused
                        if paused:
                            save_score(user_id, level, score)
                            draw_text(screen, "Paused - Press P to resume", WIDTH//2 - 100, HEIGHT//2, font)
                            pygame.display.flip()
                            # Pause loop
                            while paused:
                                for e in pygame.event.get():
                                    if e.type == pygame.KEYDOWN and e.key == pygame.K_p:
                                        paused = False

            if not paused:
                # Move snake
                head = (snake[0][0] + direction[0], snake[0][1] + direction[1])

                # Check collisions with self, wall or boundary
                if (head in snake or
                    head in walls or
                    not (0 <= head[0] < WIDTH // TILE_SIZE) or
                    not (0 <= head[1] < HEIGHT // TILE_SIZE)):
                    save_score(user_id, level, score)
                    game_over = True
                    running = False
                    continue

                # Update snake position
                snake.insert(0, head)

                if head != food:
                    snake.pop()  # Only grow when food is eaten

                else:
                    # Food eaten
                    score += 10
                    food = (random.randint(0, WIDTH // TILE_SIZE - 1), random.randint(0, HEIGHT // TILE_SIZE - 1))

                    # Level progression logic
                    if score >= 200 and level < 3:
                        level = 3
                        update_user_level(user_id, 3)
                    elif score >= 100 and level < 2:
                        level = 2
                        update_user_level(user_id, 2)

                # Draw snake
                for part in snake:
                    pygame.draw.rect(screen, (0, 255, 0), (*[x*TILE_SIZE for x in part], TILE_SIZE, TILE_SIZE))

                # Draw food
                pygame.draw.rect(screen, (255, 0, 0), (*[x*TILE_SIZE for x in food], TILE_SIZE, TILE_SIZE))

                # Draw walls
                for wall in walls:
                    pygame.draw.rect(screen, (100, 100, 100), (*[x*TILE_SIZE for x in wall], TILE_SIZE, TILE_SIZE))

            pygame.display.flip()
            clock.tick(speed)

    # Catch any unexpected error
    except Exception as e:
        print(f"An error occurred: {e}")
        pygame.quit()
        input("Press Enter to close...")

    # Game over screen
    if game_over:
        waiting_for_restart = True
        while waiting_for_restart:
            screen.fill((0, 0, 0))
            draw_text(screen, f"Game Over! Final Score: {score}", WIDTH//2 - 120, HEIGHT//2 - 20, font)
            draw_text(screen, "Press R to Restart or Q to Quit", WIDTH//2 - 140, HEIGHT//2 + 20, font)
            pygame.display.flip()

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    waiting_for_restart = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_r:
                        # Restart game with the same username
                        main(username)
                        return
                    elif event.key == pygame.K_q:
                        waiting_for_restart = False

    pygame.quit()
    input("Press Q to exit...")

# ------------------------- ENTRY POINT -------------------------
if __name__ == "__main__":
    init_db()  # Set up the database
    username = input("Enter your username: ")  # Prompt user for name
    main(username)  # Start the game
